In [48]:
import pandas as pd
import clr
import configparser

path = "data/sample/test.csv"

config = configparser.ConfigParser()
config.read('config\settings.ini')

newtonsoftjson_path = config["Paths"]["Newtonsoft.Json.Dll"]
wsconnector_path = config["Paths"]["WSConnector.Dll"]

clr.AddReference(newtonsoftjson_path)
clr.AddReference(wsconnector_path)

from WSConnector import Connector

connector = Connector()


In [11]:
df_temp = pd.read_csv(path, header=4)
df_temp = df_temp[["SN"]]
df_temp

,SN
0,S224800005
1,S224800005
2,S224800005
3,S224800005
4,S224800005
5,S224800005


In [12]:
df_temp["Measurement"] = float("NaN")
df_temp

,SN,Measurement
0,S224800005,NaN
1,S224800005,NaN
2,S224800005,NaN
3,S224800005,NaN
4,S224800005,NaN
5,S224800005,NaN


In [52]:
# read the CSV file and store it in a DataFrame
df = pd.read_csv(path, header=4)

# add a new column "Measurement" and initialize it with NaN values
df["Measurement"] = ""
df["PartNumber"] = ""
# loop through each row of the DataFrame
for i, row in df.iterrows():
    if row["Status"] == "Failed":
        measurement = row["FirstFail"]
        # search for the measurement in the header
        measurement_column = df.columns.get_loc(measurement)
        # write the value of the measurement in the new column "Measurement"
        df.at[i, "Measurement"] = df.iloc[i, measurement_column]
    serial = row["SN"]
    serial_partnumber = ""
    connector.CIMP_PartNumberRef(serial,1, serial_partnumber)
    df.at[i, "PartNumber"] = serial_partnumber

# select the desired columns
df['Only_Date'] = df['Date'].str.split(" ").str[0]
df = df[["SN", "PartNumber", 'Date', "Only_Date", "CycleTime", "Status", "FirstFail", "Measurement"]]




df

,SN,PartNumber,Date,Only_Date,CycleTime,Status,FirstFail,Measurement
0,S224800005,X13651735001+DA-1,23/01/2023 02:57:31 p. m.,23/01/2023,198.66s,Failed,UUT Current Measurment,0.183029
1,S230405749,X13651735001+DA-1,23/01/2023 03:01:49 p. m.,23/01/2023,192.42s,Failed,1.35 VDC Measurment,1.35107
2,S230400038,X13651735001+DA-1,23/01/2023 03:05:25 p. m.,23/01/2023,192.60s,Passed,NaN,
3,S214600912,X13651735001+DA-1,23/01/2023 03:18:01 p. m.,23/01/2023,198.97s,Passed,NaN,
4,S214600912,X13651735001+DA-1,23/01/2023 03:21:47 p. m.,23/01/2023,198.96s,Failed,VDC w/7 VAC Power,1.564014
5,S230503894,X13651735001+DA-1,23/01/2023 03:26:07 p. m.,23/01/2023,189.78s,Failed,VDC w/18 VAC Power,23.79631


In [14]:
header_df = {'SN': [], 'Date': [], 'CycleTime': [], 'Status': [], 'FirstFail': [], 'Measurement': []}
main_df = pd.DataFrame(header_df)

# save the new DataFrame to a new CSV file
main_df.to_csv("data/sample/output.csv", index=False)
main_df

,SN,Date,CycleTime,Status,FirstFail,Measurement
